# Image classification 

In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

### Dataset preprocessing 

In [14]:
dataset_path = "data_set"
img_size = (224, 224)
batch_size = 8

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=5,
    zoom_range=0.1,
    brightness_range=[0.9, 1.1]
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 53 images belonging to 5 classes.
Found 12 images belonging to 5 classes.


### Model building

In [15]:
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False  # Freeze base model

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


### Model training

In [16]:
model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 454ms/step - accuracy: 0.4078 - loss: 1.4015 - val_accuracy: 0.6667 - val_loss: 0.7675
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 268ms/step - accuracy: 0.7887 - loss: 0.6116 - val_accuracy: 0.9167 - val_loss: 0.2898
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 271ms/step - accuracy: 0.9512 - loss: 0.2264 - val_accuracy: 1.0000 - val_loss: 0.1891
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 261ms/step - accuracy: 1.0000 - loss: 0.1385 - val_accuracy: 0.9167 - val_loss: 0.3034
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 259ms/step - accuracy: 0.9855 - loss: 0.0987 - val_accuracy: 0.9167 - val_loss: 0.1559
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 485ms/step - accuracy: 1.0000 - loss: 0.0803 - val_accuracy: 1.0000 - val_loss: 0.1237
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 750ms/step - accuracy: 1.0000 - loss: 0.0471 - val_accuracy: 1.0000 - val_loss: 0.0829
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 764ms/step - accuracy: 1.0000 - loss: 0.0269 - val_accuracy: 1.0000 - val_loss:

In [17]:
model.save("youtube_page_classifier.h5")

### Prediction fuction

In [18]:
def predict_image(image_path):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    confidence = np.max(prediction)
    class_labels = list(train_data.class_indices.keys())
    print(f"Predicted Page Type: {class_labels[class_idx]} (Confidence: {confidence:.2f})")
    return class_labels[class_idx]

### Result

In [31]:
predict_image("test9.png")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted Page Type: search (Confidence: 0.90)


'search'